# Show MEG in infinite homogeneous and spherically symmetric head model

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import lfpykit
from equidistal_surface_points import return_equidistal_xyz
from brainsignals import plotting_convention

Start with simple current dipole moment,
set up sensor locations:

In [ ]:
# test with simple current dipole moment
p_rad = np.array([[0., 0., 1e6]]).T # 1e6 nA mu m = 1 nA m

p_tan = np.array([[np.sqrt(0.5)*1e6, np.sqrt(0.5)*1e6, 0.]]).T # 1e6 nA mu m = 1 nA m
# p_tan = np.array([[0, 1e6, 0.]]).T # 1e6 nA mu m = 1 nA m
# p_tan = np.array([[1e6, 0., 0.]]).T # 1e6 nA mu m = 1 nA m


# current dipole location in space
r_p = np.array([0, 0, 88000]) # mu m

head_radius = 100000. # mu m

num_elecs_ = 10000

x_meg_, y_meg_, z_meg_ = return_equidistal_xyz(num_elecs_, head_radius+5000) # adding 5 mm to ensure that we are outside the head

upper_idxs = np.where(z_meg_ > 0)

x_meg = x_meg_[upper_idxs]
y_meg = y_meg_[upper_idxs]
z_meg = z_meg_[upper_idxs]

meg_coords = np.vstack((x_meg, y_meg, z_meg)).T


Set up class for infinite homogeneous head:

In [ ]:
# infinite homogeneous volume conductor model
meg_infty = lfpykit.eegmegcalc.InfiniteHomogeneousVolCondMEG(sensor_locations=meg_coords)

# linear mapping between current dipole moment and signal at sensor locations
M_infty = meg_infty.get_transformation_matrix(dipole_location=r_p)

Set up class for spherically symmetric head:

In [ ]:
# spherically symmetric volume conductor model
meg_sphere = lfpykit.eegmegcalc.SphericallySymmetricVolCondMEG(r=meg_coords)

# linear mapping between current dipole moment and signal at sensor locations
M_sphere = meg_sphere.get_transformation_matrix(r_p=r_p)

Compute H and B for both head models

In [ ]:
# compute H with spherical model from radial and tangential dipole
H_sphere_prad = M_sphere @ p_rad # [nA/µm]
H_sphere_ptan = M_sphere @ p_tan # [nA/µm]
# compute H with inf hom model from radial and tangential dipole
H_infty_prad = M_infty @ p_rad # [nA/µm]
H_infty_ptan = M_infty @ p_tan # [nA/µm]

# mulitply by µ_0 to get B
mu_0 = 4*np.pi*10**-7 # [Tm/A]
# nA/µm * Tm/A = mT (millitesla)
# multiply by scaling factor to get femtoTesla
k = 1e12 # from mT to fT
B_sphere_prad = mu_0*H_sphere_prad*k # [fT]
B_sphere_ptan = mu_0*H_sphere_ptan*k # [fT]

B_infty_prad = mu_0*H_infty_prad*k # [fT]
B_infty_ptan = mu_0*H_infty_ptan*k # [fT]

Find radial and tangential parts of B

In [ ]:
def get_spherical_components(S, coords):
    """decompose S in r, theta and phi direction (fast)"""
    r = np.linalg.norm(coords, axis=1)
    theta = np.arccos(coords[:, 2] / r)
    phi = np.arctan2(coords[:, 1], coords[:, 0])
    
    # linear rotation matrix:
    M = np.array([np.r_[[np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)]].T,
                  np.r_[[np.cos(theta) * np.cos(phi), np.cos(theta) * np.sin(phi), -np.sin(theta)]].T,
                  np.r_[[-np.sin(phi), np.cos(phi), np.zeros(phi.size)]].T])
    M = np.swapaxes(M, 1, 0) # hackery
    S_sphere = M @ S
    return S_sphere[:, 0].flatten(), S_sphere[:, 1].flatten(), S_sphere[:, 2].flatten()

In [ ]:
def plot_simple_head_model(ax, radius, x, y):

    circle_npts = 100
    head_x = radius * np.cos(np.linspace(0, 2 * np.pi, circle_npts))
    head_y = radius * np.sin(np.linspace(0, 2 * np.pi, circle_npts))
    patches = []
    right_ear = mpatches.FancyBboxPatch([radius + 5000, -15000], 3000, 30000,
        boxstyle=mpatches.BoxStyle("Round", pad=5000))
    patches.append(right_ear)

    left_ear = mpatches.FancyBboxPatch([-radius - 8000, -15000], 3000, 30000,
        boxstyle=mpatches.BoxStyle("Round", pad=5000))
    patches.append(left_ear)

    collection = PatchCollection(patches, facecolor='none', edgecolor='k', alpha=1.0)
    ax.add_collection(collection)
    ax.plot(head_x, head_y, 'k')
    ax.plot([radius])
    # ax.plot(0, radius + 4700, '^', c='k', ms=15, mfc='none')
    ax.plot([-10000, 0, 10000], [radius, radius + 10000, radius], 'k')

In [ ]:

# B in direction of spherical unit vectors
B_rad_infty_prad, B_theta_infty_prad, B_phi_infty_prad = get_spherical_components(B_infty_prad, meg_coords)
B_rad_sphere_prad, B_theta_sphere_prad, B_phi_sphere_prad = get_spherical_components(B_sphere_prad, meg_coords)

vmax_meg = 100
vmin_meg = -vmax_meg

levels_meg = np.linspace(vmin_meg-5, vmax_meg+5, 22)
cbarticks = np.linspace(vmin_meg, vmax_meg, 11)

# plot 
plt.close('all')
fig = plt.figure(figsize=(6, 3.7))
fig.subplots_adjust(left=0.1,
                    bottom=0.12, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05, 
                    hspace=0.05)

ax_B_rad_infty = fig.add_subplot(231, xticks=[], yticks=[], aspect=1,
                     title=r"$B_{r}$")
ax_B_theta_infty = fig.add_subplot(232, xticks=[], yticks=[], aspect=1,
                     title=r"$B_{\theta}$")
ax_B_phi_infty = fig.add_subplot(233, xticks=[], yticks=[], aspect=1,
                     title=r"$B_{\phi}$")

ax_B_rad_sphere = fig.add_subplot(234, xticks=[], yticks=[], aspect=1,
                     title="")
ax_B_theta_sphere = fig.add_subplot(235, xticks=[], yticks=[], aspect=1,
                     title="")
ax_B_phi_sphere = fig.add_subplot(236, xticks=[], yticks=[], aspect=1,
                     title="")

for ax in fig.axes:
    ax.clear()
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_frame_on(False)

    plot_simple_head_model(ax, head_radius, x_meg, y_meg)

ax_B_rad_infty.tricontourf(x_meg, y_meg, B_rad_infty_prad, levels_meg, cmap="bwr", extend='both')
ax_B_theta_infty.tricontourf(x_meg, y_meg, B_theta_infty_prad, levels_meg, cmap="bwr", extend='both')
ax_B_phi_infty.tricontourf(x_meg, y_meg, B_phi_infty_prad, levels_meg, cmap="bwr", extend='both')

ax_B_rad_sphere.tricontourf(x_meg, y_meg, B_rad_sphere_prad, levels_meg, cmap="bwr", extend='both')
ax_B_theta_sphere.tricontourf(x_meg, y_meg, B_theta_sphere_prad, levels_meg, cmap="bwr", extend='both')
m = ax_B_phi_sphere.tricontourf(x_meg, y_meg, B_phi_sphere_prad, levels_meg, cmap="bwr", extend='both')

# illustrate radial dipole
for ax in fig.axes[:6]:
    # ax.quiver([[0]], [[0]], p_tan[0], p_tan[1], color='k', scale=1E7)
    ax.plot(0, 0, 'o', mfc='none', mec='k')
    ax.plot(0, 0, '.', mfc='k', mec='none')


# axis cross
ax = fig.axes[0]
ax.annotate("", xy=(-100E3, -90E3),
             xycoords='data', xytext=(-60E3, -90E3), textcoords='data',
             arrowprops=dict(arrowstyle="<|-",
                             connectionstyle="arc3,rad=0", facecolor='black'),
             zorder=0)
ax.annotate("", xy=(-100E3, -90E3),
             xycoords='data', xytext=(-100E3, -50E3), textcoords='data',
             arrowprops=dict(arrowstyle="<|-",
                             connectionstyle="arc3,rad=0", facecolor='black'),
             zorder=0)
ax.plot(-100E3, -90E3, 'o', ms=8, mec='k', mfc='w', zorder=1)
ax.plot(-100E3, -90E3, 'k.', ms=5, zorder=2)
ax.text(-70E3, -110E3, '$x$')
ax.text(-120E3, -60E3, '$y$')
ax.text(-120E3, -110E3, '$z$')

ax_B_rad_infty.set_title(r"$B_{r}$")
ax_B_theta_infty.set_title(r"$B_{\theta}$")
ax_B_phi_infty.set_title(r"$B_{\phi}$")

ax_B_rad_infty.set_ylabel('infinite homogeneous\nhead model')
ax_B_rad_sphere.set_ylabel('spherical\nhead model')
# colorbar MEG
cax = fig.add_axes([0.1, 0.065, 0.8, 0.01])
cbar = fig.colorbar(m, cax=cax,
                    extend='both', orientation='horizontal')
cbar.outline.set_visible(False)
cbar.set_ticks(cbarticks)
cbar.set_label(r'$B$ (fT)', labelpad=1.)

plt.savefig('Fig-3.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
# B in direction of spherical unit vectors
B_rad_infty_ptan, B_theta_infty_ptan, B_phi_infty_ptan = get_spherical_components(B_infty_ptan, meg_coords)
B_rad_sphere_ptan, B_theta_sphere_ptan, B_phi_sphere_ptan = get_spherical_components(B_sphere_ptan, meg_coords)

# plot 
plt.close('all')
fig = plt.figure(figsize=(6, 3.7))
fig.subplots_adjust(left=0.1,
                    bottom=0.12, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.05, 
                    hspace=0.05)

ax_B_rad_infty = fig.add_subplot(231, xticks=[], yticks=[], aspect=1,
                     title=r"${\bf B}_{r}$")
ax_B_theta_infty = fig.add_subplot(232, xticks=[], yticks=[], aspect=1,
                     title=r"${\bf B}_{\theta}$")
ax_B_phi_infty = fig.add_subplot(233, xticks=[], yticks=[], aspect=1,
                     title=r"${\bf B}_{\phi}$")

ax_B_rad_sphere = fig.add_subplot(234, xticks=[], yticks=[], aspect=1,
                     title="")
ax_B_theta_sphere = fig.add_subplot(235, xticks=[], yticks=[], aspect=1,
                     title="")
ax_B_phi_sphere = fig.add_subplot(236, xticks=[], yticks=[], aspect=1,
                     title="")

for ax in fig.axes:
    ax.clear()
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_frame_on(False)

    plot_simple_head_model(ax, head_radius, x_meg, y_meg)

ax_B_rad_infty.tricontourf(x_meg, y_meg, B_rad_infty_ptan, levels_meg, cmap="bwr", extend='both')
ax_B_theta_infty.tricontourf(x_meg, y_meg, B_theta_infty_ptan, levels_meg, cmap="bwr", extend='both')
ax_B_phi_infty.tricontourf(x_meg, y_meg, B_phi_infty_ptan, levels_meg, cmap="bwr", extend='both')


ax_B_rad_sphere.tricontourf(x_meg, y_meg, B_rad_sphere_ptan, levels_meg, cmap="bwr", extend='both')
ax_B_theta_sphere.tricontourf(x_meg, y_meg, B_theta_sphere_ptan, levels_meg, cmap="bwr", extend='both')
m = ax_B_phi_sphere.tricontourf(x_meg, y_meg, B_phi_sphere_ptan, levels_meg, cmap="bwr", extend='both')

# draw arrow in direction of dipole
for ax in fig.axes[:6]:
    ax.quiver([[0]], [[0]], p_tan[0], p_tan[1], color='k', scale=5E6)


# axis cross
ax = fig.axes[0]
ax.annotate("", xy=(-100E3, -90E3),
             xycoords='data', xytext=(-60E3, -90E3), textcoords='data',
             arrowprops=dict(arrowstyle="<|-",
                             connectionstyle="arc3,rad=0", facecolor='black'),
             zorder=0)
ax.annotate("", xy=(-100E3, -90E3),
             xycoords='data', xytext=(-100E3, -50E3), textcoords='data',
             arrowprops=dict(arrowstyle="<|-",
                             connectionstyle="arc3,rad=0", facecolor='black'),
             zorder=0)
ax.plot(-100E3, -90E3, 'o', ms=8, mec='k', mfc='w', zorder=1)
ax.plot(-100E3, -90E3, 'k.', ms=5, zorder=2)
ax.text(-70E3, -110E3, '$x$')
ax.text(-120E3, -60E3, '$y$')
ax.text(-120E3, -110E3, '$z$')

ax_B_rad_infty.set_title(r"$B_{r}$")
ax_B_theta_infty.set_title(r"$B_{\theta}$")
ax_B_phi_infty.set_title(r"$B_{\phi}$")

ax_B_rad_infty.set_ylabel('infinite homogeneous\nhead model')
ax_B_rad_sphere.set_ylabel('spherical\nhead model')
# colorbar MEG
cax = fig.add_axes([0.1, 0.065, 0.8, 0.01])
cbar = fig.colorbar(m, cax=cax,
                    extend='both', orientation='horizontal')
cbar.outline.set_visible(False)
cbar.set_ticks(cbarticks)
cbar.set_label(r'$B$ (fT)', labelpad=1.)

plt.savefig('Fig-4.pdf', bbox_inches='tight', pad_inches=0)